In [29]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random


In [31]:

# Importing the dataset
dataset = pd.read_csv('kidney_disease2.csv')

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 24].values


In [36]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean', skip_complete=True)
imputer = imputer.fit(X[:, :23]) # Select only relevant features for training the model
X[:, :23] = imputer.transform(X[:, :23])

TypeError: SimpleImputer.__init__() got an unexpected keyword argument 'skip_complete'

In [33]:
# Feature Selection
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(score_func=f_classif, k=10) # Select top 10 relevant features
X_new = selector.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.5, random_state =101)



ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0, max_depth=10) # Tune the hyperparameters
classifier.fit(X_train, y_train)

# Save the model
import joblib
filename ='decision_tree_model.pkl'
joblib.dump(classifier,filename)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Accuracy Score
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

## Confusion Matrix
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test, y_pred)

# Interpretation
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# ROC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, classifier.predict_proba(X_test)[:,1])
fpr, tpr, thresholds = roc_curve(y_test, classifier.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Decision Tree (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate